In [1]:
import requests
import rdflib
import warnings
warnings.filterwarnings("ignore")
from rdflib import Graph, Namespace, RDFS
from owlready2 import get_ontology
from owlready2 import *
from rdflib import Graph, Namespace, RDFS

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
#nested dict
from collections import defaultdict

g = Graph()
g.parse("SDRF_GUI/ontology/hancestro.owl", format="xml")

HANCESTRO = Namespace("http://www.hancestro.org/ontologies/hancestro#")

class_labels = defaultdict(dict)
pairs = []
all_elements = set()
for s, p, o in g.triples((None, RDFS.subClassOf, None)):
    sub_class = g.value(s, RDFS.label).toPython()
    super_class = g.value(o, RDFS.label)
    all_elements.add(sub_class)
    all_elements.add(super_class)
    if super_class:
        super_class = super_class.toPython()
    else:
        super_class = "Thing"
    pairs.append((super_class, sub_class))
    class_labels[super_class][sub_class] = class_labels[sub_class]
    if sub_class in class_labels:
        del class_labels[sub_class]

KeyboardInterrupt: 

In [12]:
d = defaultdict(dict)
root = 'ancestry category'  # first parent value
for parent, child in pairs:
    d[parent][child] = d[child]
    if root == child:
        root = parent
result = {root: d[root]}

In [13]:
def flatten(d):    
    res = []  # Result list
    if isinstance(d, dict):
        for key, val in d.items():
            res.extend(flatten(val))
    elif isinstance(d, list):
        res = d        
    else:
        raise TypeError("Undefined type for flatten: %s"%type(d))

    return res

In [15]:
ancestry_dict = result['Thing']['ancestry category']
ancestry_dict['Not available'] = {}
ancestry_dict.keys()

dict_keys(['European', 'Asian', 'African', 'Native American', 'Hispanic or Latin American', 'Greater Middle Eastern  (Middle Eastern, North African or Persian)', 'African American or Afro-Caribbean', 'Oceanian', 'uncategorised population', 'genetically isolated population', 'Aboriginal Australian', 'undefined ancestry population', 'Not available'])

In [16]:
from ParsingModule import * 
ancestry_nodes = transform_nested_dict_to_tree(ancestry_dict)
all_ancestry_elements = flatten(ancestry_dict)
store_as_gzipped_json(ancestry_dict, "ancestry_category_dict")
store_as_gzipped_json(ancestry_nodes, "ancestry_category_nodes")
store_as_gzipped_json(all_ancestry_elements, "all_ancestry_category_elements")

'Stored all_ancestry_category_elements as gzipped json'

In [8]:
def depth(d):
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [9]:
def transform_nested_dict_to_tree(d, parent_label=None, parent_value=None):
    result = []
    for key, value in d.items():
        label = key
        if parent_label:
            label = f"{key}"
        children = []
        if value:
            children = transform_nested_dict_to_tree(value, label, key)
        if children:
            result.append({"label": label, "value": key, "children": children})
        else:
            result.append({"label": label, "value": key})
    return result

In [10]:
depth(ancestry_dict['European']['Spanish'])

1

In [30]:
ancestry_dict.keys()

dict_keys(['European', 'Asian', 'African', 'Native American', 'Hispanic or Latin American', 'Greater Middle Eastern  (Middle Eastern, North African or Persian)', 'African American or Afro-Caribbean', 'Oceanian', 'uncategorised population', 'genetically isolated population', 'Aboriginal Australian', 'undefined ancestry population'])

In [34]:
ancestry_nodes = transform_nested_dict_to_tree(ancestry_dict)

In [36]:
all_ancestry_elements = flatten(ancestry_dict)

In [37]:
#save dict as pickle
import pickle
with open('C:\\Users\\tinec\\OneDrive - UGent\\git\\SDRF_StreamlitApp\\onto_dicts\\ancestry_dict.pickle', 'wb') as handle:
    pickle.dump(ancestry_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# make streamlit tree
with open('C:\\Users\\tinec\\OneDrive - UGent\\git\\SDRF_StreamlitApp\\onto_dicts\\ancestry_nodes.pickle', 'wb') as handle:
    pickle.dump(ancestry_nodes, handle, protocol=pickle.HIGHEST_PROTOCOL)
# flatten into a list of elements
with open('C:\\Users\\tinec\\OneDrive - UGent\\git\\SDRF_StreamlitApp\\onto_dicts\\all_ancestry_elements.pickle', 'wb') as handle:
    pickle.dump(all_ancestry_elements, handle, protocol=pickle.HIGHEST_PROTOCOL)

# NCBITaxon


In [4]:
# read first 20 lines of ncbitaxon owl file as txt file to determine the namespace
with open('C:\\Users\\tinec\\OneDrive - UGent\\git\\SDRF_GUI\\ontology\\ncbitaxon.owl', 'r') as f:
    for i in range(50):
        print(f.readline())

<?xml version="1.0"?>

<rdf:RDF xmlns="http://purl.obolibrary.org/obo/ncbitaxon.owl#"

     xml:base="http://purl.obolibrary.org/obo/ncbitaxon.owl"

     xmlns:obo="http://purl.obolibrary.org/obo/"

     xmlns:owl="http://www.w3.org/2002/07/owl#"

     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"

     xmlns:xml="http://www.w3.org/XML/1998/namespace"

     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"

     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"

     xmlns:terms="http://purl.org/dc/terms/"

     xmlns:oboInOwl="http://www.geneontology.org/formats/oboInOwl#"

     xmlns:ncbitaxon="http://purl.obolibrary.org/obo/ncbitaxon#">

    <owl:Ontology rdf:about="http://purl.obolibrary.org/obo/ncbitaxon.owl">

        <owl:versionIRI rdf:resource="http://purl.obolibrary.org/obo/ncbitaxon/2023-02-24/ncbitaxon.owl"/>

        <terms:description>An ontology representation of the NCBI organismal taxonomy</terms:description>

        <terms:license rdf:resource="https://creativ

In [5]:
#nested dict
from collections import defaultdict

g = Graph()
g.parse("SDRF_GUI/ontology/ncbitaxon.owl", format="xml")

class_labels = defaultdict(dict)
pairs = []
all_elements = set()
for s, p, o in g.triples((None, RDFS.subClassOf, None)):
    sub_class = g.value(s, RDFS.label).toPython()
    super_class = g.value(o, RDFS.label)
    all_elements.add(sub_class)
    all_elements.add(super_class)
    if super_class:
        super_class = super_class.toPython()
    else:
        super_class = "Thing"
    pairs.append((super_class, sub_class))
    class_labels[super_class][sub_class] = class_labels[sub_class]
    if sub_class in class_labels:
        del class_labels[sub_class]

KeyboardInterrupt: 